In [50]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

# Plot settings
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
%matplotlib inline
%config InlineBackend.figure_format='retina'


In [51]:
G=1

# Myiamoto-Nagai Disk

## Mass of the galaxy from the mass of the halo
Moster et all provides an empirical relation: $\dfrac{M_{star}}{M_{halo}} = 2N \left[\left(\dfrac{M_{halo}}{M_1}\right)^{-\beta} + \left(\dfrac{M_{halo}}{M_1}\right)^\gamma\right]^{-1}$, where $N$, $M_1$, $\beta$ and $\gamma$ are just parameters whose estimates can be found in the paper.

In [52]:
def M_1(z):
    M_10 = 11.590
    M_11 = 1.195
    return 10 ** (M_10 + M_11 * z / (z + 1))

def N_(z):
    N_10 = 0.0351
    N_11 = -0.0247
    return N_10 + N_11 * z / (z + 1)

def beta(z):
    beta_10 = 1.376
    beta_11 = -0.826
    return beta_10 + beta_11 * z / (z + 1)

def gamma(z):
    gamma_10 = 0.608
    gamma_11 = 0.329
    return gamma_10 + gamma_11 * z / (z + 1)

def m_star_m_halo(M, z=0):
    a = (M / M_1(z)) ** (-beta(z))
    b = (M / M_1(z)) ** (gamma(z))
    return 2 * M * N_(z) / (a + b)

In [53]:
#stiamo facendo tutto in int units considerando G=1 e massa in masse solari e raggi in kpc
num_particles = 20000

M_H_sun = 10**12
M_D_sun = m_star_m_halo(M_H_sun)

print(f'Mass Disk {M_D_sun:.2e}')

Mass Disk 3.43e+10


In [54]:
#proportion to rescale the masses in internal units

M_H = 1000
M_D = M_H * M_D_sun / M_H_sun

print(f'Mass Disk (int units){M_D:.5e}')

m_particles = float(M_D/num_particles)

Mass Disk (int units)3.42752e+01


## Positions and velocities of a perturber in an eccentric orbit

### Potenziale dell'alone:
$$
\Phi_{\text{halo}}(R, z) = - \frac{G M_H}{\sqrt{R^2 + z^2} + a_H}
$$

### Potenziale del disco:
$$
\Phi_{\text{disk}}(R, z) = - \frac{G M_D}{\sqrt{\left(a_D + \sqrt{z^2 + b^2}\right)^2 + R^2}}
$$

### Derivata del potenziale rispetto a \(R\) per la velocità circolare (z=0):
$$
\frac{\partial \Phi}{\partial R} =\frac{\partial (\Phi_{\text{disk}}+\Phi_{\text{halo}})}{\partial R} = G R \left( M_D \left[\left(a_D + b\right)^2 + R^2\right]^{-\frac{3}{2}} + M_H \frac{1}{\left(R + A\right)^2 R} \right)
$$

### Velocità circolare:
$$
v_{\text{circ}}(R) = \sqrt{R \cdot \frac{\partial \Phi}{\partial R}}
$$

### Distribuzione di v_z

$$
f(v_z) = \frac{1}{\sqrt{2\pi} \sigma_z} \exp\left(-\frac{v_z^2}{2\sigma_z^2}\right)
$$

### Rho Halo, Hernquist
$$
\rho(R, z) = \frac{M}{2\pi} \frac{a}{\sqrt{R^2 + z^2} \left(\sqrt{R^2 + z^2} + a\right)^3}
$$



### Densità di Miyamoto-Nagai
$$
\rho(R, z) = \frac{b^2 M}{4 \pi} \frac{a R^2 + (a + 3\sqrt{z^2 + b^2})(a + \sqrt{z^2 + b^2})^2}{\left[R^2 + (a + \sqrt{z^2 + b^2})^2\right]^{5/2}(z^2 + b^2)^{3/2}}
$$

In [55]:
const = 1.0
a = 3
b = 0.05 * a

R_max = 30  # kpc
z_max = 5  # esempio di limite massimo per z

a_H = 6

In [56]:
def rho_D(R, z):
    ''' Densità volumica di Miyamoto-Nagai  '''
    sqrt_zb = np.sqrt(z**2 + b**2)
    numerator = b**2 * M_D * (a * R**2 + (a + 3 * sqrt_zb) * (a + sqrt_zb)**2)
    denominator = 4 * np.pi * (R**2 + (a + sqrt_zb)**2)**(5/2) * (z**2 + b**2)**(3/2)
    return numerator / denominator


In [74]:
def pot_halo (R,z):
    r = np.sqrt(R**2 + z**2)
    return - G * M_H / (r + a_H)

def pot_disk (R,z):
    return - G * M_D / np.sqrt((a+np.sqrt(z**2. + b**2.))**2. + R**2.)

def dphi_dr (R):
    ''' derivata prima del potenziale totale (halo + disk) a z=0'''
    return G * R * (M_D * ((a + b)**2. + R**2)**(-3./2.) + M_H * (R+a_H)**(-2) * R**(-1))

def second_dphi_dr (R):
    ''' derivata seconda del potenziale totale (halo + disk) a z=0'''
    return dphi_dr(R)/R + G * R * (M_D*(-3*R*((a+b)**2 + R**2)**(-5/2)) - M_H * (2 * (R+a_H)**(-3) * R**(-1) + (R+a_H)**(-2) * R**(-2)))

def v_circ(R):
    return np.sqrt(dphi_dr(R) * R)

def omega(R):
    return np.sqrt(dphi_dr(R)/R)

def kappa(R):
    return np.sqrt(second_dphi_dr(R) + 3 * dphi_dr(R)/R)

def rho_H(R, z):
    ''' Densità volumica di hernquist  '''
    return (M_H / (2 * np.pi)) * (a / (np.sqrt(R**2 + z**2) * (np.sqrt(R**2 + z**2) + a)**3))

def rho_tot(R,z):
    return rho_D(R,z) + rho_H(R,z)
'''
def sigma_r(R, Q=1.2):
    S = Sigma(R)
    k = kappa(R)
    return 3.36 * Q * G * S / k'''

def sigma_z(R_values, b=b):
    print('b=',b)
    surface_density = Sigma(R_values)
    return np.sqrt(np.pi * G * surface_density * b / 2)

def compute_Q (R, b=b):
    k = kappa(R)
    S = Sigma(R)
    print('b=',b)
    return k/3.36 * np.sqrt((np.pi*b)/(G*S*2))

def Sigma(R_values):
    ''' Calcola la densità superficiale per ogni R dato'''
    # Preallocazione dell'array per i risultati
    Sigma_values = np.zeros_like(R_values)
    
    # Calcola l'integrale per ciascun valore di R
    for i, R in enumerate(R_values):
        Sigma_values[i] = 2 * np.trapz([rho_D(R, z) for z in np.linspace(0, z_max, 100)], dx=float(z_max/100))
    
    return Sigma_values

 #### Eccentric orbit

 Consider the initial position as the apoastro of the orbit and fix the eccentricity of the system.

- $ r_a = 5.2 $
- $ e = 0.65 $
- $ r_p = r_a \frac{1-e}{1+e} = 1.1$

For semplicity we consider the perturber in an orbit with z = 0.
Now consider the energy per unit mass of the perturber:
$$
    E = \frac{v_\theta ^ 2}{2} + \Phi_\text{tot} (R)
$$

The angular momentum per unit mass is 
$$
    L_z = R v_\theta
$$

We can use the conservation of angular momentum to write the velocity at the periastro as a function of the velocity at the apoastro
$$
    R_a v_{\theta a} = R_p v_{\theta p} 
$$ 

$$
       v_{\theta p} =  \frac{R_a v_{\theta a}}{R_p}  
$$

Now the conservation of enerhìgy to find an equation for $v_{\theta a}$
$$
     \frac{v_{\theta a} ^ 2}{2} + \Phi_\text{tot} (R_a) =  \frac{v_{\theta p} ^ 2}{2} + \Phi_\text{tot} (R_p)
$$
$$
     \frac{v_{\theta a} ^ 2}{2} + \Phi_\text{tot} (R_a) =  \frac{ (\frac{R_a v_{\theta a}}{R_p} ) ^ 2}{2} + \Phi_\text{tot} (R_p)
$$
$$
   v_{\theta a}^2 (1-\frac{R_a^2}{R_p^2})   = 2 ( \Phi_\text{tot} (R_a) +  \Phi_\text{tot} (R_p) )
$$
$$
     v_{\theta a} = \sqrt{2 \frac{\Phi_\text{tot} (R_p) -  \Phi_\text{tot} (R_a)}{1-\frac{R_a^2}{R_p^2}}}
$$


In [69]:
def vel_apo (a, e, z=0):
    p = a * (1-e) / (1+e)
    pot_apo = pot_disk(a,z) + pot_halo(a,z)
    pot_peri = pot_disk(p,z) + pot_halo(p,z)
    print(pot_apo, pot_peri, a/p)

    return np.sqrt(2*((pot_peri - pot_apo)/(1-(a/p)**2)))

In [70]:
#choose z,R,phi
z_pert = 0
R_pert = 5.2 #it's the apoastro
phi_pert = np.pi/3.
#compute x,y
x_pert = [R_pert * np.cos(phi_pert)]
y_pert = [R_pert * np.sin(phi_pert)]

ecc = 0.65
m_pert = m_particles*500/2

In [71]:
v_phi_pert = vel_apo (R_pert, ecc)

-94.9233721860714 -151.0545670777727 4.714285714285714


In [72]:
v_x = -v_phi_pert * np.sin(phi_pert)
v_y = v_phi_pert * np.cos(phi_pert)
v_z = 0

In [27]:
print('vx=', v_x, '\nvy=', v_y)

vx= -1.0282236417270334 
vy= 0.5936451963382402


In [73]:
print('vx=', v_x, '\nvy=', v_y)

vx= -1.9917239403003186 
vy= 1.1499223530838114


In [29]:
vcirc = v_circ(R_pert)

In [31]:
vcirc * np.cos(phi_pert)

3.3755859842475258

### Treecode parameters

In [47]:
from scipy.integrate import quad
import numpy as np

def period_orbit(r_apo, e, v, z=0):
    r_peri = r_apo * (1 - e) / (1 + e)
    # Energia specifica E
    E = 0.5 * v**2 + pot_disk(r_apo, z) + pot_halo(r_apo, z)
    print(f"E: {E}, r_peri: {r_peri}, r_apo: {r_apo}")
    
    # Momento angolare specifico L_z
    Lz = r_apo * v
    print(f"Lz: {Lz}")
    
    # Funzione per il calcolo di dr/dt
    def integrand(r):
        arg =np.abs (2 * (E - pot_disk(r, 0) - pot_halo(r, 0)) - (Lz**2 / r**2))
        if arg < 0:
            print(f"Warning: Negative argument at r={r}, arg={arg}")
            return np.inf  # Gestione dell'errore
        return 1.0 / np.sqrt(arg)
    
    # Integrazione numerica da r_peri a r_apo
    integral, _ = quad(integrand, r_peri, r_apo)
    print(f"Integral result: {integral}")
    
    # Periodo totale (fattore 2 per completare l'orbita)
    T = 2 * integral
    return T


In [48]:
T = period_orbit(R_pert, ecc, vcirc)

E: -24.302578058913525, r_peri: 1.103030303030303, r_apo: 5.2
Lz: 35.10609423617426
Integral result: 0.7497573988565343


In [49]:
T

1.4995147977130685

In [133]:
x_tot.shape

(20001,)

In [150]:
#volume = np.pi * a**2 * (np.max(z_particles)*2)
volume = (M_D+m_pert)  / np.max(rho_D(np.array(R_tot), np.array(z_tot)))
softening = ((volume/num_particles)**(1/3))

print(f'Softening parameter = {softening:.3f}')

Softening parameter = 0.095


In [151]:
def calculate_accelerations(R, z):
    # Potenziale dell'alone
    denominator_halo = (R**2 + z**2 + softening**2)**1.5
    acc_R_halo = -G * M_H * R / denominator_halo
    acc_z_halo = -G * M_H * z / denominator_halo

    # Potenziale del disco
    sqrt_zb2 = np.sqrt(z**2 + b**2)
    denominator_disk = ((a + sqrt_zb2)**2 + R**2)**1.5
    acc_R_disk = -G * M_D * R / denominator_disk
    acc_z_disk = -G * M_D * z * (a + sqrt_zb2) / (sqrt_zb2 * denominator_disk)

    # Accelerazioni totali
    acc_R_total = acc_R_halo + acc_R_disk
    acc_z_total = acc_z_halo + acc_z_disk
    return acc_R_total, acc_z_total

# Calcolo delle accelerazioni
acc_R, acc_z = calculate_accelerations(np.array(R_tot), np.array(z_tot))

# Modulo dell'accelerazione totale per ogni particella
acc_total = np.sqrt(acc_R**2 + acc_z**2)

# Accelerazione massima
acc_max = np.max(acc_total)

print(f"L'accelerazione massima è: {acc_max:.5e}")

L'accelerazione massima è: 4.17693e+04


In [152]:
dt = 0.5 * np.sqrt(softening / acc_max)
print('Timestep:', dt)

Timestep: 0.0007539262017918349


In [153]:
rho_mean = np.mean(rho_D(np.array(R_tot), np.array(z_tot)))
t_dyn = np.sqrt(1/rho_mean)

print(f'Dynamical timescale = {t_dyn}')

Dynamical timescale = 1.8810655297313683


In [154]:
M_D

34.27515185606664

In [ ]:
def w_onfile(filename, num_points, masses, dimension, x, y, z, v_x, v_y, v_z):
    # Scegli un indice casuale
    #random_number= 11022
    modified_masses = [masses] * num_points
    # Modifica la massa della particella alla posizione random_number
   # modified_masses[random_number] = 1000 * masses
    #print(modified_masses[random_number])
    with open(filename, 'w') as file:
        file.write(f'  {num_points} \n  {dimension} \n  {0}  \n')
        for mass in modified_masses:
            file.write(f'  {mass}  \n')
        for i in range(num_points):
            file.write(f'  {x[i]}  {y[i]}  {z[i]}  \n')
        for i in range(num_points):
            file.write(f'  {v_x[i]}  {v_y[i]}  {v_z[i]} \n')
            
filename = 'treecode_in_perturber.txt'

w_onfile(filename, (num_particles+1), M_D/num_particles, 3, x_tot, y_tot, z_tot, v_x, v_y, v_z) 

In [141]:
# ./treecode in=../treecode_in_perturber.txt out=../treecode_out_perturber.txt dtime=0.00077 eps=0.096 theta=0.2 tstop=22.8 dtout=0.47 > ../output_treecode_perturber.txt